In [1]:
import folium

In [2]:
from pyproj import crs
import pandas as pd

from folium.plugins import HeatMap

from folium.plugins import MarkerCluster

from folium import IFrame, FeatureGroup, LayerControl

from folium.plugins import GroupedLayerControl

# Création de la carte interactive

In [3]:
Tree = pd.read_excel('data/20230717_Lacuzon Arbre_data.xlsx')

In [4]:
Tree[Tree["Génération"]==1]

,Unnamed: 0,ID,Sosa,Génération,Name,Genre,BIRTH_PLACE,BIRTH_DATE,MARRIAGE_PLACE,MARRIAGE_DATE,...,BIRTH_YEAR Réel ou Projeté,BIRTH_YEAR,DEATH_YEAR Réel ou Projeté,DEATH_YEAR,Age,Lieu normé Réel ou Projeté,Lieux_normé,Département,long,lat
0,0,@I0024@,1,1,Benoit PROST,M,NaN,NaN,NaN,NaN,...,R,1990,NaN,NaN,NaN,R,"Paris, France",75.0,2.352222,48.856614


In [5]:
# Create a Map instance
m = folium.Map(location=[48.8639368381152, 2.349781581911858], zoom_start=6, control_scale=True)


In [8]:
# Définition de la foction pour afficher la popup
def HtmlPopup(location_info):
    html = """ \
    <h5>"""+location_info["Name"]+"""</h5><p style="line-height: 200%">"""
    
    html += str(location_info["BIRTH_YEAR"]) + """ (""" + str(location_info["BIRTH_YEAR Réel ou Projeté"])
    
    html += """) - """

    html += str(location_info["DEATH_YEAR"]) + """ (""" + str(location_info["DEATH_YEAR  Réel ou Projeté"])
    
    html += """)</p>"""
    
    return html

In [9]:
GenerationList = Tree["Génération"].drop_duplicates().sort_values().values

GenerationLayer = []

for i in range(len(GenerationList)):
    Gene = GenerationList[i]
    globals()[f"marker_cluster_G{Gene}"] = MarkerCluster(name=f"G{Gene}", overlay=True).add_to(m)
   
    GenerationLayer.append(globals()[f"marker_cluster_G{Gene}"])
    for index, location_info in Tree[Tree["Génération"]==Gene].iterrows():
        location = [location_info["lat"], location_info["long"]]
        
        popup = HtmlPopup(location_info)
        
        folium.Marker(
            location=location,
            popup=popup,
        ).add_to(globals()[f"marker_cluster_G{Gene}"])
    
LayerControl().add_to(m)

GroupedLayerControl(
    groups={'Génération': GenerationLayer},
    exclusive_groups=True,
    collapsed=False,
).add_to(m)

m

In [155]:
outfp = "GeneMap.html"
m.save(outfp)

# Parser du Gedcom

***Objectif*** : Travailler un gedcom afin de créer un dataframe panda exploitable

In [82]:
from ged4py import GedcomReader
from ged4py.date import DateValueVisitor

import pandas as pd


In [7]:
# Path to your `.ged` file
path = 'data/20230713_Prost Lacuzon-Export_all.ged'

# Initialize the parser
parser = GedcomReader(path, encoding="utf-8")


In [198]:
# Fonction ajout individu

def ParseIndividuGeneration(ID, Sosa, Genera):
    for i, indi in enumerate(parser.records0("INDI")):
        if indi.xref_id == ID:

            IndividuGénération['ID'].append(indi.xref_id)
            IndividuGénération['Sosa'].append(Sosa)
            IndividuGénération['Génération'].append(Genera)
            IndividuGénération['Name'].append(indi.name.format())
            IndividuGénération['Genre'].append(indi.sub_tag_value("SEX"))
            IndividuGénération['BIRTH_PLACE'].append(indi.sub_tag_value("BIRT/PLAC"))
            IndividuGénération['BIRTH_DATE'].append(indi.sub_tag_value("BIRT/DATE"))
            IndividuGénération['MARRIAGE_PLACE'].append(indi.sub_tag_value("MARR/PLAC"))
            IndividuGénération['MARRIAGE_DATE'].append(indi.sub_tag_value("MARR/DATE"))
            IndividuGénération['DEATH_PLACE'].append(indi.sub_tag_value("DEAT/PLAC"))
            IndividuGénération['DEATH_DATE'].append(indi.sub_tag_value("DEAT/DATE"))        


            if indi.father != None:
                IndividuGénération['Father'].append(indi.father.xref_id) 
            else: IndividuGénération['Father'].append(None)

            if indi.mother != None:
                IndividuGénération['Mother'].append(indi.mother.xref_id)
            else: IndividuGénération['Mother'].append(None)
            break

In [251]:
# Dictionnaire des individus par génération
IndividuGénération = {'ID':[], 'Sosa':[], "Génération" :[] ,"Name" :[], "Genre" :[], "BIRTH_PLACE" :[], "BIRTH_DATE" :[], "MARRIAGE_PLACE" :[], "MARRIAGE_DATE" :[], "DEATH_PLACE" :[], "DEATH_DATE" :[], "Father" :[], "Mother" :[]}



In [252]:
# Initialisation du sujet 0

ID = '@I0024@'
Sosa = 1
Genera = 1
            
ParseIndividuGeneration(ID, 1, 1)

In [253]:

DfIndividuGénération = pd.DataFrame.from_dict(IndividuGénération)

In [254]:
#Boucle itération génération
Genera = 1
Gen = True

while Gen == True:
    Gen = False
    
    #Rajout père
    for index, DfIndividu_info in DfIndividuGénération[DfIndividuGénération['Génération']==Genera].iterrows():
        if DfIndividu_info['Father'] != None:
            ID = DfIndividu_info['Father']
            Sosa = DfIndividu_info['Sosa'] *2
            GeneraInd = Genera + 1

            ParseIndividuGeneration(ID, Sosa, GeneraInd)
            Gen = True

    #Rajout mère
    for index, DfIndividu_info in DfIndividuGénération[DfIndividuGénération['Génération']==Genera].iterrows():
        if DfIndividu_info['Mother'] != None:
            ID = DfIndividu_info['Mother']
            Sosa = DfIndividu_info['Sosa'] *2 + 1
            GeneraInd = Genera + 1

            ParseIndividuGeneration(ID, Sosa, GeneraInd)
            Gen = True
    
    DfIndividuGénération = pd.DataFrame.from_dict(IndividuGénération)
    Genera += 1

In [258]:
#Suppression des doublons
DfIndividuGénération= DfIndividuGénération.drop_duplicates(subset=['ID'])

In [259]:
DfIndividuGénération.to_excel("Lacuzon Arbre.xlsx")

# Geolocalisation

In [310]:
from geopy.geocoders import GoogleV3

import requests, json

In [311]:

# Opening JSON file
f = open("Param/param.json")

# returns JSON object as 
# a dictionary
param = json.load(f)


In [312]:
# Création Geocoder Google
locator = GoogleV3(api_key=param['GooglePlateform_API']['api_key'])


In [316]:
dFLieu = pd.read_excel('Data/Lieu Genealogie.xlsx')

In [344]:
Lieu_Info = {'Lieu':[], 'location':[], 'long':[], 'lat':[]}

for index, DfLieu_info in dFLieu.iterrows():
    Lieu_Info['Lieu'].append(DfLieu_info.values[0])
    location = locator.geocode(DfLieu_info.values[0])
    
    if location == None:
        Lieu_Info['location'].append(None)
        Lieu_Info['long'].append(None)
        Lieu_Info['lat'].append(None)      
    else:
        Lieu_Info['location'].append(location)
        Lieu_Info['long'].append(location.longitude)
        Lieu_Info['lat'].append(location.latitude)

In [347]:
DfLieu_Info = pd.DataFrame.from_dict(Lieu_Info)

In [349]:
DfLieu_Info.to_excel("LieuGeoCode.xlsx")